In [2]:
from pyspark import SparkContext
sc = SparkContext(master='local[4]')


In [3]:
A = sc.parallelize(range(10000))
print(A.getNumPartitions())

4


In [5]:
B=A.repartition(10)
print (B.getNumPartitions())

10


In [6]:
C = sc.parallelize(range(100000), numSlices=5)
print (C.getNumPartitions())

5


In [7]:
A=sc.parallelize(range(1000000)).map(lambda x:(x,x)).partitionBy(10)
print(A.glom().map(len).collect())

[100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]


In [8]:
B = A.filter(lambda x:x[0]%5==0)
print (B.glom().map(len).collect())

[100000, 0, 0, 0, 0, 100000, 0, 0, 0, 0]


In [12]:
C = B.map(lambda x:(x[1]/10, x[1])).partitionBy(10)
print(C.glom().map(len).collect())

[20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000]


In [16]:
def getInfo(G):
    d=0
    if len(G)>1: 
        for i in range(len(G)-1):
            d+=abs(G[i+1][1]-G[i][1]) # access the glomed RDD that is now a  list
        return (G[0][0],len(G),d)
    else:
        return(None)
    
print(B.take(5))
output = B.glom().map(lambda B:getInfo(B)).collect()
print (output)

[(0, 0), (10, 10), (20, 20), (30, 30), (40, 40)]
[(0, 100000, 999990), None, None, None, None, (5, 100000, 999990), None, None, None, None]
